In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# from sutils import *
import os, json
from glob import glob

import tensorflow as tf

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten,Input, BatchNormalization, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.3.0
2.0.8


In [3]:
for entry in glob('/Data/*'):
    print entry

In [4]:
aircraft_df = pd.DataFrame(np.load('Data/aircraft carrier.npy'))
airplane_df = pd.DataFrame(np.load('Data/airplane.npy'))
alarm_df = pd.DataFrame(np.load('Data/alarm clock.npy'))
ambulance_df = pd.DataFrame(np.load('Data/ambulance.npy'))

df_list =  [aircraft_df, airplane_df, alarm_df, ambulance_df]
X = pd.concat(df_list)
X.shape

(539530, 784)

In [5]:
y_target = []
for i, df in enumerate(df_list):
    labels = np.zeros([df.shape[0],4])
    labels[:,i] = 1
    if i == 0:
        y_target = labels
    else:
        y_target = np.vstack([y_target,labels])        

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_target, test_size=0.33, random_state=42)

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [7]:
X_train = X_train.values.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.values.reshape(X_test.shape[0],28,28,1)

In [8]:
Inp = Input(shape=(28,28,1),name = 'Input_layer')

#ConvBlock 01
conv01 = Conv2D(32, (3, 3), padding='same',activation = 'relu', input_shape=Inp.shape,name = 'Conv01_layer')(Inp)
conv02 = Conv2D(32, (3, 3),activation = 'relu',name = 'Conv02_layer')(conv01)
maxpool_01 = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool01_layer')(conv02)
drop01 = Dropout(0.25,name = 'Dropout01_layer')(maxpool_01)

#Convblock 02
conv03 = Conv2D(64, (3, 3), padding='same',activation = 'relu',name = 'Conv03_layer')(drop01)
conv04 = Conv2D(64, (3, 3),activation = 'relu',name = 'Conv04_layer')(conv03)
maxpool_02 = MaxPooling2D(pool_size=(2, 2),name = 'MaxPool02_layer')(conv04)
drop02 = Dropout(0.25,name = 'Dropout02_layer')(maxpool_02)

# Fully Connected Dense block
x = Flatten(name = 'Flatten_layer')(drop02)
x = Dense(512, activation='relu',name = 'Dense01_layer')(x)
x = Dropout(0.5,name = 'Dropout03_layer')(x)
logits_layer = Dense(y_train.shape[1], name= 'logits_layer')(x)
output = Activation('softmax',name = 'Sofftmax_layer')(logits_layer)

In [9]:
# Define model inputs and output
model = Model(Inp, output)

In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input_layer (InputLayer)     (None, 28, 28, 1)         0         
_________________________________________________________________
Conv01_layer (Conv2D)        (None, 28, 28, 32)        320       
_________________________________________________________________
Conv02_layer (Conv2D)        (None, 26, 26, 32)        9248      
_________________________________________________________________
MaxPool01_layer (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
Dropout01_layer (Dropout)    (None, 13, 13, 32)        0         
_________________________________________________________________
Conv03_layer (Conv2D)        (None, 13, 13, 64)        18496     
_________________________________________________________________
Conv04_layer (Conv2D)        (None, 11, 11, 64)        36928     
__________

In [11]:
# initiate RMSprop optimizer
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [12]:
print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    rotation_range=30,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False)  # randomly flip images

Using real-time data augmentation.


In [13]:
# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

In [ ]:
batch_size = 32
num_classes = 10
epochs = 50
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'cifar10_simple_CNN.h5'

In [ ]:
hist = model.fit_generator(datagen.flow(X_train, y_train,
                                 batch_size=batch_size),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    epochs=epochs,
                    validation_data=(X_test, y_test),
                    workers=4)

Epoch 1/50
11296/11296 [==============================] - 1912s - loss: 0.5124 - acc: 0.8083 - val_loss: 0.3108 - val_acc: 0.8894
Epoch 2/50
11296/11296 [==============================] - 2071s - loss: 0.3616 - acc: 0.8711 - val_loss: 0.2765 - val_acc: 0.9025
Epoch 3/50
11296/11296 [==============================] - 2104s - loss: 0.3388 - acc: 0.8799 - val_loss: 0.2667 - val_acc: 0.9058
Epoch 4/50
11296/11296 [==============================] - 2099s - loss: 0.3341 - acc: 0.8821 - val_loss: 0.2696 - val_acc: 0.9047
Epoch 5/50
11296/11296 [==============================] - 2100s - loss: 0.3340 - acc: 0.8828 - val_loss: 0.2696 - val_acc: 0.9055
Epoch 6/50
11296/11296 [==============================] - 2108s - loss: 0.3347 - acc: 0.8823 - val_loss: 0.2856 - val_acc: 0.9064
Epoch 7/50
11296/11296 [==============================] - 2094s - loss: 0.3352 - acc: 0.8820 - val_loss: 0.2749 - val_acc: 0.9023
Epoch 8/50
11296/11296 [==============================] - 2095s - loss: 0.3366 - acc: 0.88

In [ ]:
def plot_train(hist):
    h = hist.history
    if 'acc' in h:
        meas='acc'
        loc='lower right'
    else:
        meas='loss'
        loc='upper right'
    plt.plot(hist.history[meas])
    plt.plot(hist.history['val_'+meas])
    plt.title('model '+meas)
    plt.ylabel(meas)
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc=loc)
    

In [ ]:
plot_train(hist)